In [1]:
from scipy.stats import levy_stable
import numpy as np
import torch
import math
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import optim
import os
import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import StepLR

In [2]:
# a simple fully connected neural network
class simpleNet(nn.Module):

    def __init__(self, input_dim=28*28 , width=400, depth=4, num_classes=10):
        super(simpleNet, self).__init__()
        self.input_dim = input_dim 
        self.width = width
        self.depth = depth
        self.num_classes = num_classes
        
        layers = self.get_layers()

        self.fc = nn.Sequential(
            nn.Linear(self.input_dim, self.width, bias=False),
            nn.ReLU(inplace=True),
            *layers,
            nn.Linear(self.width, self.num_classes, bias=False),
        )

    def get_layers(self):
        layers = []
        for i in range(self.depth - 2):
            layers.append(nn.Linear(self.width, self.width, bias=False))
            layers.append(nn.ReLU())
        return layers

    def forward(self, x):
        x = x.view(x.size(0), self.input_dim)
        x = self.fc(x)
        return x

In [3]:
# load MNIST data
batch_size=128
data_tf = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize(mean=(0.1307,), std=(0.3081,))])


train_dataset = datasets.MNIST(root='./data', train=True, transform=data_tf, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=data_tf)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [4]:
class SGLD(Optimizer):
    """
    SGLD optimiser based on pytorch's SGD.
    Note that the weight decay is specified in terms of the gaussian prior sigma.
    """

    def __init__(self, params, lr, norm_sigma=0, addnoise=True):

        weight_decay = 1 / (norm_sigma ** 2)

        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))

        defaults = dict(lr=lr, weight_decay=weight_decay, addnoise=addnoise)

        super(SGLD, self).__init__(params, defaults)

    def step(self):
        """
        Performs a single optimization step.
        """
        loss = None

        for group in self.param_groups:

            weight_decay = group['weight_decay']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)

                if group['addnoise']:

                    langevin_noise = p.data.new(p.data.size()).normal_(mean=0, std=1) / np.sqrt(group['lr'])
                    p.data.add_(-group['lr'],
                                0.5 * d_p + langevin_noise)
                else:
                    p.data.add_(-group['lr'], 0.5 * d_p)

        return loss


In [5]:
import torch
from torch.optim import Optimizer


class PSGLD(Optimizer):
    """
    Penalized SGLD optimiser based on pytorch's SGD.
    Note that the weight decay is specified in terms of the gaussian prior sigma.
    """

    def __init__(self, params, lr, delta, constrain_list, lp = 1):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))

        defaults = dict(lr=lr, delta=delta, constrain_list = constrain_list, lp = lp)
        super(PSGLD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(PSGLD, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)
            
#     def phi0(self, t, gamma):
#         return torch.exp(-t * gamma)

#     def phi1(self, t, gamma):
#         return - 1/gamma * torch.exp(-t * gamma) + 1 / gamma

#     def phi2(self, t, gamma):
#         tmp = 1/gamma ** 2
#         tmp = tmp * (np.torch(-gamma*t)-1)
#         tmp = tmp + t/gamma
#         return tmp

#     def covariance(self, t, gamma):
#         e2gt = torch.exp(-2*gamma*t)
#         egt = torch.exp(-gamma*t)
#         a11 = - (1/(2*gamma)) * e2gt + 1/(2*gamma)
#         a12 = e2gt / (2*gamma*gamma) - egt/(gamma**2) - 1/(2*(gamma**2)) + 1/(gamma ** 2)
#         a22 = -e2gt/(2*(gamma ** 3)) + 2*egt / (gamma**3) + 1/(2 * (gamma ** 3)) - 2/(gamma ** 3) + t/(gamma ** 2)
#         tmp = torch.tensor([[a11, a12], [a12,a22]])
#         return tmp

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            delta = group['delta']
            constrain_list = group['constrain_list']
            lp = group['lp']
            
            idx = 0
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                norm_ord = torch.norm(p.data, p = lp)
                constrain = constrain_list[idx]
                if norm_ord > constrain:
                    g1 = torch.pow(torch.abs(p.data)/norm_ord, lp-1)
                    tmp_con = (norm_ord-constrain) * (g1 * torch.sign(p.data))
                    d_p.add_(delta, tmp_con)
                

                p.data.add_(-group['lr'], d_p)
                noise = torch.normal(0,1,size = p.data.shape).cuda()
                p.data.add_(torch.sqrt(torch.tensor(2*group['lr'])), noise)
                idx += 1

        return loss


In [6]:
lr_list = [3*(0.1)**7]
delta = 500

# 1-norm of the result from SGLD, used as the constraints in the constrained network
norm_list = torch.tensor([94749.414 , 48327.09  , 48311.508 ,  1030.1956])
# we can change the value of s here
constrain_list = norm_list * 0.8

# select 1-norm
lp = 1
epoch = 400
# calculate the averaged result from 5 runs
N = 5

# path to save models
PATH = './MNIST-3FCN-Pen/constrain08/SGLD-lp1/'
try:
    os.mkdir(PATH)
except OSError as exc:
    pass


train_acc_all = []

for n in range(N):
    print(n)
    for i in range(len(lr_list)):
        learning_rate = lr_list[i]
        trainErrorList=[]
        trainAccList=[]

    
        model = simpleNet()
        if torch.cuda.is_available():
                model = model.cuda()
        criterion = nn.functional.nll_loss
#         optimizer = SGLD(model.parameters(), lr = learning_rate, norm_sigma = 1, addnoise = True)
        optimizer = PSGLD(model.parameters(), lr=learning_rate, delta = delta, constrain_list = constrain_list, lp = lp)
        scheduler = StepLR(optimizer, step_size=100, gamma=0.9)
        
        for l in range(epoch):
            train_acc=0
            for data in train_loader:
                img, label = data
                img=img.view(img.size(0),-1)
                if torch.cuda.is_available():
                    img = img.cuda()
                    label = label.cuda()
                else:
                    img = Variable(img)
                    label = Variable(label)
                out = model(img)
                loss = F.cross_entropy(out, label, reduction='sum')
                print_loss = loss.data.item()
                _, pred = torch.max(out.data, 1)
                train_acc += pred.eq(label.view_as(pred)).sum().item()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            trainErrorList.append(loss.data.item())
            trainAccList.append(train_acc/60000)
            scheduler.step()
            if l % 100 == 0 and l !=0:
                print(train_acc/60000)
            if (train_acc/60000) >=0.7 and train_acc/60000 == max(trainAccList):
                tmp_path = PATH + '/model' + '{}'.format(n) +'.pth'
                torch.save(model, tmp_path)
        train_acc_all.append(trainAccList)
        print(train_acc/60000)

0


C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type simpleNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


0.7912166666666667
0.8519166666666667
0.8799833333333333
0.89265
1
0.7782166666666667
0.8503
0.8776666666666667
0.8885
2
0.7739666666666667
0.8504666666666667
0.8769166666666667
0.8854333333333333
3
0.7845833333333333
0.84895
0.8788666666666667
0.8858166666666667
4
0.7784333333333333
0.85865
0.88205
0.8950666666666667


In [8]:
norms = []
norm_cal = []
for p in model.parameters():
#         if p.grad is None:
#             continue
#         else:
    norm_ord = torch.norm(p.data, p = lp).detach()
    norm_cal.append(norm_ord)
norms.append(norm_cal)
norm = np.linalg.norm(norms, ord = lp, axis = 1)
print(np.max(norm)/ torch.norm(torch.tensor([94749.414 , 48327.09  , 48311.508 ,  1030.1956]),p = 1))

tensor(0.7999, device='cuda:0')
